In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
# 카메라의 면적 구하기

angle_degrees_1 = 57.5
angle_degrees_2 = 71.37796
h = 1100
angle_radians_1 = math.radians(angle_degrees_1)
angle_radians_2 = math.radians(angle_degrees_2)

x = 2 * h / math.tan(angle_radians_1)
print(x)

y = 2 * h / math.tan(angle_radians_2)
print(y)

# degree = math.atan2(1100,(x/2)) * 180 / math.pi
# print(degree)

S = x*y
S/(1920*1080)

In [38]:
def calculate_distance(point1, point2):
    # 두 점을 numpy 배열로 변환
    point1 = np.array(point1)
    point2 = np.array(point2)
    
    # 두 점 사이의 차이 계산
    difference = point2 - point1
    
    # 유클리드 거리 계산
    distance = np.linalg.norm(difference)
    
    return distance

# 두 점의 좌표 설정
point_heli = (25416,131.100006,32661.3301)  # 여기에 실제 좌표값을 넣으세요
point_cam = (25434,129.460007,31561.3008)  # 여기에 실제 좌표값을 넣으세요

# 거리 계산
distance = calculate_distance(point_heli, point_cam)
print(distance)

1100.177781295008


거리에 따른 픽셀 관계

In [117]:
# 유니티상에서의 Vector3 거리를 계산(카메라와 헬기 간 거리 계산 용 블록)
# 카메라 2 1.1km 위치: (25434,129.4599,31560.6504)
# 카메라 2 550 m 위치: (25434,129.4599,32110.4004)
# 카메라 3 275 m 위치: (25434,129.4599,32386.4004)

# 2번 헬기의 z값 목록
# 3000: 34561.25 (2번 헬기의 경우 뒷산에 가려지므로 카메라 position.z를 300061.25으로 변경하고
# 헬기의 z는 1500m의 33061.2로 설정)

# 1500: 33061.2
# 1400: 32960.6
# 1300: 32860.6
# 1200: 32761.25
# 1100: 32661.33
# 1000: 32561.62
# 900: 32461.47 
# 800: 32361.4  
# 700: 32261.4 
# 600: 32161.3
# 500: 32061.3
# 400: 31961.3
# 300: 31861.3
# 200: 31761
# 100: 31660

# 50: 31608

point_heli = (25416,131.100006,32960.7)        # 헬기
point_cam = (25434,129.460007,31560.65)       # 카메라
distance = math.sqrt((point_cam[0] - point_heli[0])**2 + (point_cam[1] - point_heli[1])**2 + (point_cam[2] - point_heli[2])**2)
print("두 점 사이의 거리:", distance)



두 점 사이의 거리: 1400.166665828289


이미지 마스킹을 통해 비 영역을 제거한 픽셀 검출용 영상 제작(비 오는 상황)

In [18]:
# 테스트 마스킹 이미지 생성용 코드

rain_image = cv2.imread("../../data/vtd/EO/heavy_rain/land/temp/rain_1.png")
# snow_image = cv2.imread("../../data/vtd/EO/snow/temp/snow_1.png")
# forest_image = cv2.imread("../../data/vtd/EO/forest/temp/0.png")
heli_image = cv2.imread("../../data/vtd/EO/heavy_rain/land/temp/heli_1.png")
mask_image = cv2.cvtColor(rain_image,cv2.COLOR_BGR2RGB)

# # 숲 배경 마스크 1
# lower_bound = np.array([30, 50, 50])
# upper_bound = np.array([100, 87, 100])
# # 숲 배경 마스크 2
# lower_2 = np.array([89, 88, 38])
# upper_2 = np.array([130, 106, 60])

upper_bound = np.array([100, 100, 90])
lower_bound = np.array([40, 40, 40])

# 눈 검은색 설정 픽셀 설정 경계 상한
# upper_bound = np.array([176, 176, 174])

mask = cv2.inRange(mask_image, lower_bound, upper_bound)
heli_image[mask != 0] = [0, 0, 0]
# mask_2 = cv2.inRange(mask_image, lower_2, upper_2)
# combined_mask = cv2.bitwise_or(mask,mask_2)
# heli_image[combined_mask != 0] = [0, 0, 0]

cv2.imwrite('../../data/vtd/EO/heavy_rain/land/temp/example_1.png', heli_image)
cv2.imshow('mask image', mask)
# cv2.imshow('mask_2 image', mask_2)
cv2.imshow('result image', heli_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
# 마우스 클릭 이벤트에 반응하여 픽셀 RGB를 찍는 콜백 함수 정의
def show_rgb(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        b, g, r = image[y, x]
        print(f"RGB value at ({x}, {y}): ({r}, {g}, {b})")

image = cv2.imread("../../data/vtd/EO/forest/final/0/06.png")
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', show_rgb)
while True:
    cv2.imshow('Image', image)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cv2.destroyAllWindows()

RGB value at (96, 201): (0, 0, 0)
RGB value at (96, 201): (0, 0, 0)
RGB value at (97, 201): (0, 0, 0)
RGB value at (100, 201): (38, 59, 70)
RGB value at (106, 199): (0, 0, 0)
RGB value at (113, 197): (0, 0, 0)
RGB value at (126, 188): (0, 0, 0)
RGB value at (138, 184): (53, 62, 66)
RGB value at (144, 181): (57, 67, 64)
RGB value at (148, 179): (46, 61, 66)
RGB value at (149, 178): (46, 61, 66)
RGB value at (153, 175): (0, 0, 0)
RGB value at (158, 172): (0, 0, 0)
RGB value at (162, 169): (0, 0, 0)
RGB value at (177, 153): (0, 0, 0)
RGB value at (182, 146): (0, 0, 0)
RGB value at (192, 133): (0, 0, 0)
RGB value at (201, 118): (0, 0, 0)
RGB value at (205, 108): (0, 0, 0)
RGB value at (208, 103): (0, 0, 0)
RGB value at (211, 94): (0, 0, 0)
RGB value at (212, 92): (0, 0, 0)
RGB value at (212, 91): (0, 0, 0)
RGB value at (209, 91): (0, 0, 0)
RGB value at (204, 92): (0, 0, 0)
RGB value at (198, 93): (0, 0, 0)
RGB value at (186, 93): (0, 0, 0)
RGB value at (182, 93): (0, 0, 0)
RGB value at (17

In [19]:
# 반복문을 통해 한 폴더 내 모든 이미지파일에 대해 마스크를 적용
# 매트랩에서 ppm 검출 전 이 코드를 실행하여 
# temp 파일을 읽고 final 파일의 알맞은 고각 폴더에 파일을 옮긴다
# 이 코드 실행 후 매트랩에서 마스킹 된 이미지에 대하여 RGB 검출 수행

rain_image = cv2.imread("../../data/vtd/EO/heavy_rain/land/temp/rain_1.png")
# snow_image = cv2.imread("../../data/vtd/EO/snow/temp/snow_1.png")
mask_image = cv2.cvtColor(rain_image, cv2.COLOR_BGR2RGB)    # 마스크로 쓸 비가 촬영된 영상
lower_bound = np.array([40, 40, 40])
upper_bound = np.array([100, 100, 90])                     # 비(혹은 눈, 배경)의 RGB값을 검출할 경계 설정
mask = cv2.inRange(mask_image, lower_bound, upper_bound)
source_folder = "../../data/vtd/EO/heavy_rain/land/temp/90"             # 헬기 이미지가 저장된 위치, 폴더 변경하며 코드 실행, temp 폴더에 유니티에서 찍은 헬기사진이 들어있을것
output_folder = "../../data/vtd/EO/heavy_rain/land/final/90"              # 마스킹 된 이미지가 저장될 위치

# 눈 배경일 시  
upper_bound = np.array([177, 177, 177])                     # 비(혹은 눈, 배경)의 RGB값을 검출할 경계 설정

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

for i in range(19):
    filename = f"{i:02}.png"    # 00.png, 01.png, ..., 18.png 등의 이름 형식 이미지들에 대해 반복문 수행
    heli_image_path = os.path.join(source_folder, filename)
    heli_image = cv2.imread(heli_image_path)
    heli_image[mask != 0] = [0, 0, 0]
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, heli_image)



In [79]:
# 반복문을 통해 한 폴더 내 모든 이미지파일에 대해 마스크를 적용
# 비, 눈과같은 고정된 마스킹 이미지 소스가 아닌 유동적으로 변하는 마스킹 이미지 소스에 대해 반복문을 수행

# forest_image = cv2.imread("../../data/vtd/EO/forest/temp/0.png")
# mask_image = cv2.cvtColor(forest_image, cv2.COLOR_BGR2RGB)    # 마스크로 쓸 비가 촬영된 영상

# 숲 배경 마스크 1
upper_bound = np.array([100, 84, 100])
lower_bound = np.array([30, 50, 60])
# 숲 배경 마스크 2
upper_2 = np.array([130, 106, 60])
lower_2 = np.array([89, 88, 38])

# 파란 계열 마스크 3
upper_3 = np.array([65, 255, 255])
lower_3 = np.array([0, 0, 0])

# mask = cv2.inRange(mask_image, lower_bound, upper_bound)
source_folder = "../../data/vtd/EO/heavy_rain/temp/0"             # 헬기 이미지가 저장된 위치, 폴더 변경하며 코드 실행, temp 폴더에 유니티에서 찍은 헬기사진이 들어있을것
output_folder = "../../data/vtd/EO/heavy_rain/final/0"              # 마스킹 된 이미지가 저장될 위치

for i in range(19):
    filename = f"{i:02}.png"    # 00.png, 01.png, ..., 18.png 등의 이름 형식 이미지들에 대해 반복문 수행

    heli_image_path = os.path.join(source_folder, filename)
    heli_image = cv2.imread(heli_image_path)

    mask = cv2.inRange(heli_image, lower_bound, upper_bound)
    mask_2 = cv2.inRange(heli_image, lower_2, upper_2)
    mask_3 = cv2.inRange(heli_image, lower_3, upper_3)
    combined_mask = cv2.bitwise_or(mask,mask_2)
    combined_mask = cv2.bitwise_or(combined_mask,mask_3)

    heli_image[combined_mask != 0] = [0, 0, 0]
    
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, heli_image)

# cv2.imshow("mask_2",mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [56]:
# # 마스크 따로 쓰기 테스트용 코드

# # 파란색 제거 마스크 1
# upper_bound = np.array([65, 255, 255])
# lower_bound = np.array([0, 0, 0])

# # mask = cv2.inRange(mask_image, lower_bound, upper_bound)
# source_folder = "../../data/vtd/EO/forest/final/0"             # 헬기 이미지가 저장된 위치, 폴더 변경하며 코드 실행, temp 폴더에 유니티에서 찍은 헬기사진이 들어있을것
# output_folder = "../../data/vtd/EO/forest/final/0"              # 마스킹 된 이미지가 저장될 위치

# for i in range(19):
#     filename = f"{i:02}.png"    # 00.png, 01.png, ..., 18.png 등의 이름 형식 이미지들에 대해 반복문 수행

#     heli_image_path = os.path.join(source_folder, filename)
#     heli_image = cv2.imread(heli_image_path)

#     mask = cv2.inRange(heli_image, lower_bound, upper_bound)

#     heli_image[mask != 0] = [0, 0, 0]
    
#     output_path = os.path.join(output_folder, filename)
#     cv2.imwrite(output_path, heli_image)

# cv2.imshow("mask",mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [31]:
# 헬기 명도 매핑값
# 여기에 하늘, 4계절 데이터 포함하면 됨
print(3.1098 * 43 /255)
print(3.1098 * 47 /255)
print(3.1098 * 50 /255)
print(3.1098 * 54 /255)
print(3.1098 * 57 /255)
print(3.1098 * 61 /255)
print(3.1098 * 64 /255)
print(3.1098 * 68 /255)
print(3.1098 * 71 /255)
print(3.1098 * 75 /255)
print(3.1098 * 78 /255)
print(3.1098 * 82 /255)
# 배경 온도 매핑값
print(3.1098 * 30 /255)

0.5243976470588235
0.5731788235294117
0.609764705882353
0.6585458823529411
0.6951317647058823
0.7439129411764706
0.7804988235294117
0.82928
0.8658658823529412
0.9146470588235294
0.9512329411764705
1.000014117647059
0.36585882352941174


In [47]:
# 배경온도 값을 명도로 매핑(명도가 포토샵처럼 -100부터 100까지인 경우)
# Blender의 0~1까지 범위일 것

value = 0.4878

# Mapping from 0 to 1 range to -100 to 100 range
mapped_value = (value * 200) - 100
mapped_value

-2.4399999999999977